<h1>賃貸情報のスクレイピング</h1>
<h3>・各属性の一覧<br>name：賃貸名<br>address：住所<br>ward：区<br>station1~3：最寄り駅<br>age：築年数<br>floor：階数<br>rent：賃料<br>admin：管理費<br>plan：間取り<br>area：面積<br>deposit：敷金<br>gratuity：礼金</h3>

In [76]:
from bs4 import BeautifulSoup
import datetime
import pandas as pd
from pandas import Series, DataFrame
import re
import requests
import time

In [102]:
t_now = datetime.datetime.now()
print(t_now)

2021-09-16 14:55:46.054129


In [101]:
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50"

result = requests.get(url)
content = result.content
soup = BeautifulSoup(content)

In [103]:
# ページ数の取得
pages_set = soup.find('div', {'class': 'pagination_set-nav'}).text
pages_split = pages_set.split()
page = int(pages_split[5])

In [104]:
# 全ページのurlを取得
urls = [url]

for num in range(2, page+1):
    page_url = url + '&=' + str(num)
    urls.append(page_url)

In [116]:
name = []
address = []
station1 = []
station2 = []
station3 = []
age = []
floor = []
rent = []
admin = []
plan = []
area = []
deposit = []
gratuity = []

In [117]:
for url in urls:
    result = requests.get(url)
    content = result.content
    soup = BeautifulSoup(content)
    house_list = soup.find('div', {'id': 'js-bukkenList'})
    cassette_items = house_list.find_all('div', {'class': 'cassetteitem'})

    for cassette in cassette_items:
        # 掲載部屋数
        n_room = len(cassette.find_all('tbody'))
        # 賃貸名
        house_name = str(cassette.find('div', {'class': 'cassetteitem_content-title'}).text)
        # 住所
        house_address = str(cassette.find('li', {'class': 'cassetteitem_detail-col1'}).text)
        # 最寄り駅のリスト
        station_list = cassette.find_all('div', {'class', 'cassetteitem_detail-text'})
        # 築年数
        text_age = cassette.find('li', 'cassetteitem_detail-col3').text.strip().split()[0]
        if text_age == '新築':
            house_age = 0
        else:
            house_age = int(text_age.replace('築', '').replace('年', ''))
        
        for i in range(n_room):
            name.append(house_name)
            address.append(house_address.replace('東京都', ''))
            station1.append(station_list[0].text)
            station2.append(station_list[1].text)
            station3.append(station_list[2].text)
            age.append(house_age)
        
        
        body = cassette.find_all('tbody')
        
        for i in range(n_room):
            # 部屋情報
            room_info = body[i].find_all('td')
            
            # 階数
            house_floor = room_info[2].text.strip().replace('階', '').replace('-', '0').replace('B', '-')
            # 複数階にまたがっているとき、最上階のみ取得
            if '0' in house_floor and len(house_floor) != 1:
                house_floor = house_floor[-1]
            floor.append(int(house_floor))
            
            # 賃料
            rent.append(float(room_info[3].find_all('li')[0].text.strip('万円')))
            
            # 管理費
            house_admin = room_info[3].find_all('li')[1].text.strip('円')
            if house_admin == '-':
                house_admin = 0
            admin.append(int(house_admin))
            
            # 敷金
            house_deposit = room_info[4].find_all('li')[0].text.strip('万円')
            if house_deposit == '-':
                house_deposit = 0
            deposit.append(float(house_deposit))
            
            # 礼金
            house_gratuity = room_info[4].find_all('li')[1].text.strip('万円')
            if house_gratuity == '-':
                house_gratuity = 0
            gratuity.append(float(house_gratuity))
            
            # 間取り
            plan.append(room_info[5].find_all('li')[0].text)
            
            # 面積
            if room_info[5].find_all('li')[1].text == '':
                house_area = 0.0
            else:
                house_area = room_info[5].find_all('li')[1].text
            area.append(float(re.sub('m2', '', house_area)))
        
time.sleep(10)

In [118]:
# 区
ward = []

for i in range(len(address)):
    s = ''
    for j in range(len(address[i])):
        if (address[i][j] == '区'):
            s += '区'
            break;
        s += address[i][j]
    ward.append(s)

In [120]:
name = Series(name)
address = Series(address)
ward = Series(ward)
station1 = Series(station1)
station2 = Series(station2)
station3 = Series(station3)
age = Series(age)
floor = Series(floor)
rent = Series(rent)
admin = Series(admin)
plan = Series(plan)
area = Series(area)
deposit = Series(deposit)
gratuity = Series(gratuity)

In [148]:
# 取得データをcsvファイルに保存
housing_df = pd.concat([name, address, ward, station1, station2, station3, age, floor, rent, admin, plan, area, deposit, gratuity], axis=1)
housing_df.columns = ['name', 'address', 'ward', 'station1', 'station2', 'station3', 'age', 'floor', 'rent', 'admin', 'plan', 'area', 'deposit', 'gratuity']
housing_df.to_csv('housing_23ward.csv', index=False)

In [154]:
df = pd.read_csv('housing_23ward.csv')
df.head()

,name,address,ward,station1,station2,station3,age,floor,rent,admin,plan,area,deposit,gratuity
0,クレヴィアリグゼ日暮里,荒川区東日暮里６,荒川区,ＪＲ山手線/日暮里駅 歩6分,ＪＲ山手線/西日暮里駅 歩14分,ＪＲ常磐線/日暮里駅 歩6分,4,5,11.0,10000,1DK,25.01,11.0,0.0
1,クレヴィアリグゼ日暮里,荒川区東日暮里６,荒川区,ＪＲ山手線/日暮里駅 歩6分,ＪＲ山手線/西日暮里駅 歩14分,ＪＲ常磐線/日暮里駅 歩6分,4,6,11.0,10000,1DK,25.01,11.0,0.0
2,クレヴィアリグゼ日暮里,荒川区東日暮里６,荒川区,ＪＲ山手線/日暮里駅 歩6分,ＪＲ山手線/西日暮里駅 歩14分,ＪＲ常磐線/日暮里駅 歩6分,4,3,11.2,10000,1LDK,25.05,11.2,0.0
3,クレヴィアリグゼ日暮里,荒川区東日暮里６,荒川区,ＪＲ山手線/日暮里駅 歩6分,ＪＲ山手線/西日暮里駅 歩14分,ＪＲ常磐線/日暮里駅 歩6分,4,6,11.2,10000,1DK,25.01,11.2,0.0
4,クレヴィアリグゼ日暮里,荒川区東日暮里６,荒川区,ＪＲ山手線/日暮里駅 歩6分,ＪＲ山手線/西日暮里駅 歩14分,ＪＲ常磐線/日暮里駅 歩6分,4,8,11.5,10000,1LDK,25.05,11.5,0.0
